## Section 2

### Question1

In [1]:
import collections
import numpy as np
import util

In [2]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    return message.lower().split()

In [3]:
def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    word_count = collections.defaultdict(int)

    for message in messages:
        words = set(get_words(message))  # فقط یک‌بار برای هر پیام
        for word in words:
            word_count[word] += 1

    dictionary = {}
    for word, count in word_count.items():
        if count >= 5:  # آستانه تعریف‌شده: حداقل در ۵ پیام ظاهر شده باشد
            dictionary[word] = len(dictionary)

    return dictionary

In [4]:
def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    matrix = np.zeros((len(messages), len(word_dictionary)), dtype=int)

    for i, message in enumerate(messages):
        words = get_words(message)
        for word in words:
            if word in word_dictionary:
                j = word_dictionary[word]
                matrix[i, j] += 1

    return matrix

### Question 2

In [5]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """
    n_messages, n_words = matrix.shape

    # تقسیم داده‌ها بر اساس برچسب کلاس
    spam_indices = labels == 1
    ham_indices = labels == 0

    # تعداد پیام‌های هر کلاس
    n_spam = np.sum(spam_indices)
    n_ham = np.sum(ham_indices)

    # احتمال اولیه کلاس‌ها (با log)
    log_class_priors = np.log(np.array([
        n_ham / len(labels),
        n_spam / len(labels)
    ]))

    # مجموع کلمات در هر کلاس + لاپلاس
    spam_word_counts = np.sum(matrix[spam_indices], axis=0) + 1
    ham_word_counts = np.sum(matrix[ham_indices], axis=0) + 1

    # مجموع کل کلمات + |V| (برای لاپلاس)
    spam_total = np.sum(spam_word_counts)
    ham_total = np.sum(ham_word_counts)

    # احتمال شرطی کلمات برای هر کلاس (با log)
    log_word_probs = np.vstack([
        np.log(ham_word_counts / ham_total),  # کلاس 0 = ham
        np.log(spam_word_counts / spam_total)  # کلاس 1 = spam
    ])

    return {
        "log_class_priors": log_class_priors,
        "log_word_probs": log_word_probs
    }



In [7]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    log_class_priors = model["log_class_priors"]
    log_word_probs = model["log_word_probs"]  # شکل: [2 × V]

    # ضرب لگاریتم احتمال‌ها در بردار تعداد کلمات
    log_likelihood_ham = matrix @ log_word_probs[0] + log_class_priors[0]
    log_likelihood_spam = matrix @ log_word_probs[1] + log_class_priors[1]

    # انتخاب کلاسی که لگاریتم احتمالش بیشتره
    return (log_likelihood_spam > log_likelihood_ham).astype(int)


### Question 3

In [8]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    log_word_probs = model["log_word_probs"]  # شکل: [2 × V]

    # محاسبه تفاوت لگاریتمی: log(P(w | spam)) - log(P(w | ham))
    log_ratios = log_word_probs[1] - log_word_probs[0]

    # لیست کلمات به همراه score آن‌ها
    index_to_word = {index: word for word, index in dictionary.items()}
    word_scores = [(index_to_word[i], log_ratios[i]) for i in range(len(log_ratios))]

    # مرتب‌سازی نزولی بر اساس score و گرفتن top 5
    top_5 = sorted(word_scores, key=lambda x: x[1], reverse=True)[:5]

    # فقط لیست کلمات را برگردان
    return [word for word, score in top_5]




### RESULT !!

In [9]:
def main():
    train_messages, train_labels = util.load_spam_dataset('train.tsv')
    val_messages, val_labels = util.load_spam_dataset('val.tsv')
    test_messages, test_labels = util.load_spam_dataset('test.tsv')

    dictionary = create_dictionary(train_messages)

    util.write_json('dictionary', dictionary)

    train_matrix = transform_text(train_messages, dictionary)

    np.savetxt('sample_train_matrix', train_matrix[:100,:])

    val_matrix = transform_text(val_messages, dictionary)
    test_matrix = transform_text(test_messages, dictionary)

    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

    np.savetxt('naive_bayes_predictions', naive_bayes_predictions)

    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

    print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

    top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

    print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

    util.write_json('top_indicative_words', top_5_words)

if __name__ == "__main__":
    main()

Naive Bayes had an accuracy of 0.978494623655914 on the testing set
The top 5 indicative words for Naive Bayes are:  ['claim', 'won', 'prize', 'tone', 'urgent!']
